<a href="https://colab.research.google.com/github/taejjing/kaggle_LANL/blob/master/LANL_init.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import auth
auth.authenticate_user()

from google.colab import drive
drive.mount('/gdrive')

In [0]:
DATA_DIR = '/content/lanl/'

In [0]:
! mkdir ~/.kaggle
! cp /gdrive/'My Drive'/'Colab Notebooks'/kaggle/kaggle.json ~/.kaggle/kaggle.json
! chmod 600 ~/.kaggle/kaggle.json

In [0]:
! kaggle competitions download -c LANL-Earthquake-Prediction -p {DATA_DIR}
! ls -l {DATA_DIR}

100% 2.02G/2.03G [00:21<00:00, 58.5MB/s]
100% 2.03G/2.03G [00:21<00:00, 102MB/s] 
total 2372428
-rw-r--r-- 1 root root      34135 May 15 17:17 sample_submission.csv
-rw-r--r-- 1 root root  253730367 May 15 17:17 test.zip
-rw-r--r-- 1 root root 2175592057 May 15 17:18 train.csv.zip


In [0]:
# ! unzip {DATA_DIR}train.csv.zip -d {DATA_DIR}
# ! unzip {DATA_DIR}test.zip -d {DATA_DIR}test
! rm -rf {DATA_DIR}*.zip
! ls -l {DATA_DIR}

total 9331728
-rw-r--r-- 1 root root      34135 May 15 17:17 sample_submission.csv
drwxr-xr-x 2 root root      90112 May 15 17:21 test
---------- 1 root root 9555558244 Jan 10 01:12 train.csv


https://github.com/Vettejeep/MSDS696-Masters-Final-Project/blob/master/earthquake_model.ipynb

In [0]:
import os
import time
import warnings
import traceback
import numpy as np
import pandas as pd
from scipy import stats
import scipy.signal as sg
import multiprocessing as mp
from scipy.signal import hann
from scipy.signal import hilbert
from scipy.signal import convolve
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

import xgboost as xgb
import lightgbm as lgb
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

from tqdm import tqdm
warnings.filterwarnings("ignore")

In [0]:
SIG_LEN = 150000
NUM_SEG_PER_PROC = 4000
NUM_THREADS = 6

NY_FREQ_IDX = 75000  # the test signals are 150k samples long, Nyquist is thus 75k.
CUTOFF = 18000
MAX_FREQ_IDX = 20000
FREQ_STEP = 2500

TRAIN_DIR = os.path.join(DATA_DIR, 'train')
if not os.path.isdir(TRAIN_DIR) :
    os.mkdir(TRAIN_DIR)
TRAIN_LEN = 629145480

In [0]:
def split_raw_data():
    max_start_index = TRAIN_LEN - SIG_LEN
    slice_len = max_start_index // 6
    
    for i, chunk in enumerate(pd.read_csv(os.path.join(DATA_DIR, 'train.csv'), chunksize=slice_len)):
        print('working', i)
        chunk.to_csv(os.path.join(OUTPUT_DIR, 'raw_data_%d.csv' % i), index=False)

In [0]:
start = time.time()

split_raw_data()

print("{}s".format(time.time() - start))

working 0
working 1
working 2
working 3
working 4
working 5
working 6
{}s 452.410790681839
